In [1]:
import pandas as pd

In [2]:
import funciones_kwichon as f

# Datos de migraciones en España.
Extraídos del INE (del 2006 al 2021, microdatos)

I M P O R T A N T E: Descargar la carpeta 'datos' que contiene los archivos con las migraciones de: 
https://drive.google.com/drive/folders/1QGW4sFkmz6wWSkeby4oPHDzKO6STyRiB?usp=sharing

## Diccionario de los códigos del tamaño de los municipios

In [3]:
cod_tamu = {1: 'Hasta 10.000 habitantes', 
            2: 'Municipio no capital de 10.001 a 20.000',
            3 : 'Municipio no capital de 20.001 a 50.000', 
            4: 'Municipio no capital de 50.001 a 100.000',      
            5: 'Municipio no capital  de más de 100.000',        
            6: 'Municipio capital de provincia'
            
           }

## Tabla de los códigos provincias

In [4]:
cod_prov = pd.read_excel('./datos/MIGRACIONES/cod_prov.xlsx', header=1, names = ['CPRO', 'NOMBRE'])
cod_prov

,CPRO,NOMBRE
0,1,Araba/Álava
1,2,Albacete
2,3,Alicante/Alacant
3,4,Almería
4,5,Ávila
5,6,Badajoz
6,7,"Balears, Illes"
7,8,Barcelona
8,9,Burgos
9,10,Cáceres


In [5]:
cod_prov.columns

Index(['CPRO', 'NOMBRE'], dtype='object')

#### Columnas
- CPRO: Código de provincia
- NOMBRE: Nombre de la provincia

## Tabla de los códigos de los municipios

#### Hay 52 hojas en el fichero de Excel que contiene los códigos de los municipios, una por cada municipio

In [6]:
cod_muni = pd.DataFrame()
for muni in range(52):
    cod_muni= pd.concat([cod_muni,pd.read_excel('./datos/MIGRACIONES/23codmun.xlsx', sheet_name=muni, header=2)], ignore_index=True)
cod_muni

,CPRO,CMUN,DC,NOMBRE
0,1,51,3,Agurain/Salvatierra
1,1,1,4,Alegría-Dulantzi
2,1,2,9,Amurrio
3,1,49,3,Añana
4,1,3,5,Aramaio
...,...,...,...,...
8126,50,296,7,"Zaida, La"
8127,50,297,3,Zaragoza
8128,50,298,9,Zuera
8129,51,1,3,Ceuta


In [7]:
# Se elimina la columna DC que no nos aporta información para el estudio.
cod_muni.drop(columns='DC', inplace=True)

In [8]:
cod_muni.columns

Index(['CPRO', 'CMUN', 'NOMBRE '], dtype='object')

#### Columnas
- CPRO: Código de provincia
- CMUN: Código de municipio
- NOMBRE: Nombre del municipio

#### Eliminación de posibles espacios antes o después del nombre de las columnas

In [9]:
cod_muni.columns = list(map(lambda x: x.strip(),list(cod_muni.columns)))

#### Los códigos de municipio completos con los que se trabajan habitualmente son la suma del código de provicia y el código de municipio de la tabla

In [10]:
# Creación de nueva columna 'CMUNICIPIO' con el código de municipio completo

cod_muni['CMUNICIPIO'] = cod_muni.apply(lambda x: int(f"{x['CPRO']:02}" + f"{x['CMUN']:03}"), axis=1)                         
cod_muni

,CPRO,CMUN,NOMBRE,CMUNICIPIO
0,1,51,Agurain/Salvatierra,1051
1,1,1,Alegría-Dulantzi,1001
2,1,2,Amurrio,1002
3,1,49,Añana,1049
4,1,3,Aramaio,1003
...,...,...,...,...
8126,50,296,"Zaida, La",50296
8127,50,297,Zaragoza,50297
8128,50,298,Zuera,50298
8129,51,1,Ceuta,51001


#### Creación de fila con el valor CPRO 66, CMUN 666 y CMUNICIPIO 66666  para provincias y municipios que corresponde al extranjero, y con valores 0 como desconocido

In [11]:
cod_muni.loc[len(cod_muni.index)] = [66, 666, 'Extranjero', 66666]
cod_muni.loc[len(cod_muni.index)] = [0, 0, 'Desconocido', 0]
cod_muni

,CPRO,CMUN,NOMBRE,CMUNICIPIO
0,1,51,Agurain/Salvatierra,1051
1,1,1,Alegría-Dulantzi,1001
2,1,2,Amurrio,1002
3,1,49,Añana,1049
4,1,3,Aramaio,1003
...,...,...,...,...
8128,50,298,Zuera,50298
8129,51,1,Ceuta,51001
8130,52,1,Melilla,52001
8131,66,666,Extranjero,66666


## Tabla de las migraciones del 2006 al 2021

In [12]:
# Creación de una tabla para todas las migraciones desde el año 2011 al 2021

migraciones = f.migrations_tables(2006, 2021)
migraciones

,SEXO,PROVNAC,EDAD,PROVALTA,MUNIALTA,ANOVAR,PROVBAJA,MUNIBAJA,TAMUALTA,TAMUBAJA
0,1,1,2,48,020,2006,1,,6,1
1,1,1,53,1,,2006,1,059,1,6
2,1,1,0,25,,2006,25,203,1,2
3,1,1,62,66,302,2006,1,059,,6
4,1,1,65,28,,2006,28,079,1,6
...,...,...,...,...,...,...,...,...,...,...
40996119,6,66,77,66,999,2021,52,001,,4
40996120,6,66,21,66,999,2021,52,001,,4
40996121,1,66,32,66,126,2021,52,001,,4
40996122,1,66,43,66,999,2021,52,001,,4


In [13]:
migraciones.dtypes

SEXO         int64
PROVNAC      int64
EDAD         int64
PROVALTA     int64
MUNIALTA    object
ANOVAR       int64
PROVBAJA     int64
MUNIBAJA    object
TAMUALTA    object
TAMUBAJA    object
dtype: object

#### Columnas
- SEXO: 1 hombre, 6 mujer
- PROVNAC: Código de la provincia de nacimiento
- EDAD: Edad
- PROVALTA: Código de la provincia de alta
- MUNIALTA: Código del municipio de alta
- ANOVAR: Año de la migración
- PROVBAJA: Código de la provincia de baja
- MUNIBAJA: Código del municipio de baja
- TAMUALTA: Tamaño del municipio de alta (ver diccionario de códigos de municipio)
- TAMUBAJA: Tamaño del municipio de alta (ver diccionario de códigos de municipio)

#### Los valores de MUNIALTA y MUNIBAJA vacíos, se cambian al valor  0 como desconocido.
#### Se observa que aunque no tengan el código de municipio, sí tiene un valor de código de provincia y el valor del tamaño del municipio es siempre 1 (Menos de 10000 habitantes)

In [14]:
migraciones['MUNIALTA'].value_counts()

       7593628
079    2763518
999    2414215
019    1782977
666    1710526
        ...   
440          1
512          1
209          1
513          1
514          1
Name: MUNIALTA, Length: 363, dtype: int64

In [15]:
migraciones['MUNIBAJA'].value_counts()

       6632918
079    2523057
999    1956304
019    1619753
128     721839
        ...   
508          8
516          6
512          6
515          6
506          5
Name: MUNIBAJA, Length: 364, dtype: int64

In [16]:
null = migraciones['MUNIBAJA'].value_counts().index[0]
null

'   '

In [17]:
migraciones[migraciones['MUNIALTA']==null]['TAMUALTA'].value_counts()

1    7593628
Name: TAMUALTA, dtype: int64

In [18]:
migraciones[migraciones['MUNIBAJA']==null]['TAMUBAJA'].value_counts()

1    6632918
Name: TAMUBAJA, dtype: int64

In [19]:
migraciones['MUNIBAJA'] = migraciones['MUNIBAJA'].replace(null,'000').apply(int)
migraciones['MUNIALTA'] = migraciones['MUNIALTA'].replace(null,'000').apply(int)
migraciones

,SEXO,PROVNAC,EDAD,PROVALTA,MUNIALTA,ANOVAR,PROVBAJA,MUNIBAJA,TAMUALTA,TAMUBAJA
0,1,1,2,48,20,2006,1,0,6,1
1,1,1,53,1,0,2006,1,59,1,6
2,1,1,0,25,0,2006,25,203,1,2
3,1,1,62,66,302,2006,1,59,,6
4,1,1,65,28,0,2006,28,79,1,6
...,...,...,...,...,...,...,...,...,...,...
40996119,6,66,77,66,999,2021,52,1,,4
40996120,6,66,21,66,999,2021,52,1,,4
40996121,1,66,32,66,126,2021,52,1,,4
40996122,1,66,43,66,999,2021,52,1,,4


In [20]:
migraciones[migraciones['PROVALTA']==66]['MUNIALTA'].value_counts()

999    2414215
666    1710526
125     167158
110     125835
126     116428
        ...   
513          1
512          1
440          1
209          1
514          1
Name: MUNIALTA, Length: 202, dtype: int64

#### Se detectan varios valores de TAMUALTA y TAMUBAJA vacíos que coinciden con provincia 66 que es extranjero.
#### Es la población que emigra al extranjero o inmigrantes. No se tendrán en cuenta en el estudio.

In [21]:
# Se detectan varios valores de TAMUALTA y TAMUBAJA vacíos que coinciden con provincia 66 que es extranjero.
# Es la población que emigra al extranjero o inmigrantes. No se tendrán en cuenta en el estudio.
migraciones['TAMUALTA'].value_counts()

6    10201005
1     7593628
3     6007341
      5647590
4     4147679
2     3947724
5     3451157
Name: TAMUALTA, dtype: int64

In [22]:
migraciones['TAMUBAJA'].value_counts()

     9504142
6    9478261
1    6632918
3    5186063
4    3620587
2    3380442
5    3193711
Name: TAMUBAJA, dtype: int64

In [23]:
null_ta = migraciones['TAMUBAJA'].value_counts().index[0]
null_ta

' '

In [24]:
migraciones[migraciones['TAMUALTA']== null_ta]['PROVALTA'].value_counts()

66    5647590
Name: PROVALTA, dtype: int64

In [25]:
migraciones[migraciones['TAMUBAJA']== null_ta]['PROVBAJA'].value_counts()

66    9504142
Name: PROVBAJA, dtype: int64

#### Se detecta que cuando en PROVALTA y PROVBAJA se indica 66 (extranjero), hay varios códigos que corresponderían al país. Se cambia a 666 extranjero para unificar valores.

In [26]:
migraciones[migraciones['PROVBAJA']== 66]['MUNIBAJA'].value_counts()

999    1956304
128     708262
228     636287
343     470285
125     418048
        ...   
130          7
512          6
516          6
515          6
506          5
Name: MUNIBAJA, Length: 203, dtype: int64

In [27]:
migraciones[migraciones['PROVALTA']== 66]['MUNIALTA'].value_counts()

999    2414215
666    1710526
125     167158
110     125835
126     116428
        ...   
513          1
512          1
440          1
209          1
514          1
Name: MUNIALTA, Length: 202, dtype: int64

In [28]:
migraciones.loc[migraciones['PROVALTA']==66, 'MUNIALTA']  = 666

In [29]:
migraciones.loc[migraciones['PROVBAJA']==66, 'MUNIBAJA']  = 666

In [30]:
migraciones

,SEXO,PROVNAC,EDAD,PROVALTA,MUNIALTA,ANOVAR,PROVBAJA,MUNIBAJA,TAMUALTA,TAMUBAJA
0,1,1,2,48,20,2006,1,0,6,1
1,1,1,53,1,0,2006,1,59,1,6
2,1,1,0,25,0,2006,25,203,1,2
3,1,1,62,66,666,2006,1,59,,6
4,1,1,65,28,0,2006,28,79,1,6
...,...,...,...,...,...,...,...,...,...,...
40996119,6,66,77,66,666,2021,52,1,,4
40996120,6,66,21,66,666,2021,52,1,,4
40996121,1,66,32,66,666,2021,52,1,,4
40996122,1,66,43,66,666,2021,52,1,,4


#### Se añaden dos columnas con los códigos de municipio de alta y de baja.

In [31]:
migraciones['CODMUNIALTA'] = migraciones.apply(lambda x: f"{x['PROVALTA']:02}" + f"{x['MUNIALTA']:03}", axis=1)                                                                                   

In [32]:
migraciones['CODMUNIBAJA'] = migraciones.apply(lambda x: f"{x['PROVBAJA']:02}" + f"{x['MUNIBAJA']:03}", axis=1)

In [33]:
migraciones

,SEXO,PROVNAC,EDAD,PROVALTA,MUNIALTA,ANOVAR,PROVBAJA,MUNIBAJA,TAMUALTA,TAMUBAJA,CODMUNIALTA,CODMUNIBAJA
0,1,1,2,48,20,2006,1,0,6,1,48020,01000
1,1,1,53,1,0,2006,1,59,1,6,01000,01059
2,1,1,0,25,0,2006,25,203,1,2,25000,25203
3,1,1,62,66,666,2006,1,59,,6,66666,01059
4,1,1,65,28,0,2006,28,79,1,6,28000,28079
...,...,...,...,...,...,...,...,...,...,...,...,...
40996119,6,66,77,66,666,2021,52,1,,4,66666,52001
40996120,6,66,21,66,666,2021,52,1,,4,66666,52001
40996121,1,66,32,66,666,2021,52,1,,4,66666,52001
40996122,1,66,43,66,666,2021,52,1,,4,66666,52001


In [34]:
migraciones.dtypes

SEXO            int64
PROVNAC         int64
EDAD            int64
PROVALTA        int64
MUNIALTA        int64
ANOVAR          int64
PROVBAJA        int64
MUNIBAJA        int64
TAMUALTA       object
TAMUBAJA       object
CODMUNIALTA    object
CODMUNIBAJA    object
dtype: object

## Tabla de migraciones solo dentro de España (se excluyen las realizadas hacia o desde el extranjero)

Se filtra la tabla de migraciones excluyendo las filas en las que se indica la provincia de alta y de baja con código 66.

In [35]:
extranjero = ((migraciones['PROVALTA']==66) | (migraciones['PROVBAJA']==66))
solo_migr = migraciones[~extranjero]
solo_migr

,SEXO,PROVNAC,EDAD,PROVALTA,MUNIALTA,ANOVAR,PROVBAJA,MUNIBAJA,TAMUALTA,TAMUBAJA,CODMUNIALTA,CODMUNIBAJA
0,1,1,2,48,20,2006,1,0,6,1,48020,01000
1,1,1,53,1,0,2006,1,59,1,6,01000,01059
2,1,1,0,25,0,2006,25,203,1,2,25000,25203
4,1,1,65,28,0,2006,28,79,1,6,28000,28079
5,6,1,3,1,0,2006,1,59,1,6,01000,01059
...,...,...,...,...,...,...,...,...,...,...,...,...
39922030,1,66,26,50,297,2021,52,1,6,4,50297,52001
39922031,1,66,25,50,297,2021,52,1,6,4,50297,52001
39922032,1,66,45,50,297,2021,52,1,6,4,50297,52001
39922033,1,66,31,51,1,2021,52,1,4,4,51001,52001


#### Se guardan las 4 tablas en ficheros .csv para poder utilizar en otros notebooks

In [36]:
migraciones.to_csv('./csv_files/migraciones_2006_2021.csv')

In [37]:
solo_migr.to_csv('./csv_files/solo_migr_2006_2021.csv')

In [38]:
cod_muni.to_csv('./csv_files/cod_muni.csv')

In [ ]:
cod_prov.to_csv('./csv_files/cod_prov.csv')

### Link para acceder a los archivos .csv generados: https://drive.google.com/drive/folders/1sncg5KzVbL3fYQish_0YwmA5XnlicorQ?usp=sharing